<a href="https://colab.research.google.com/github/Dimildizio/chat-bot-yakutia/blob/main/bot/vk_yak_gigachat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install langchain gigachat vk_api

In [20]:
from google.colab import userdata
from typing import Callable, Dict, List
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage
import os
import vk_api
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id

In [29]:
apikey = userdata.get('gigachat_autoreq')
token = userdata.get('vk_public1')
authorize = vk_api.VkApi(token=token)
longpoll = VkLongPoll(authorize)
os.environ["GIGACHAT_CREDENTIALS"] = apikey
giga = GigaChat(profanity=False, temperature=0.2,  verify_ssl_certs=False)

In [9]:
description = f"""Мы общаемся по поводу работы Администрации города в Якутске. Твоя задача вежливо и официальным тоном дать информацию на запрос
если он касается работы государственных органов или проблем региона Якутск. Вежливо игнорируй оскорбительные запросы и запросы с нецензурной лексикой"""


system_message = SystemMessage(content=(
        f"""{description}

Контекст: Вы - виртуальный ассистент, созданный для помощи в коммуникациях между населением Республики Саха (Якутия) и местной администрацией. Ваша задача - обрабатывать запросы от жителей, касающиеся различных аспектов жизни в регионе, включая муниципальные вопросы, социальные программы, инфраструктурные проекты и прочее. Вы должны предоставлять точную и актуальную информацию, соблюдая официальный и уважительный тон в общении.

Особенности региона: Якутия - крупнейший регион России с уникальными климатическими условиями и культурным наследием. Большая часть запросов может касаться особенностей жизни в суровых климатических условиях, развития социальной инфраструктуры и поддержки коренных народов Севера.

Типы запросов:

    Информация о муниципальных услугах (водоснабжение, отопление, дорожное строительство).
    Вопросы социальной поддержки (льготы, социальные выплаты).
    Информация о местных культурных и образовательных событиях.
    Вопросы, связанные с экологией и природопользованием.
    Обращения по вопросам местного самоуправления и участия в общественной жизни.

Тон общения: Официальный, уважительный, информативный. Важно демонстрировать понимание запросов и предоставлять исчерпывающие ответы, основанные на актуальной нормативно-правовой документации и официальной информации администрации.

Источники информации: Официальный сайт администрации Республики Саха (Якутия), муниципальные порталы, официальные публикации и пресс-релизы.
"""))


In [ ]:

class DialogueAgent:
    def __init__(self, name: str, system_message: SystemMessage, model: giga) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}:"
        self.reset()

    def reset(self):
        self.message_history = ["Вот разговор:"]

    def send(self) -> str:
        """
        Applies the chat model to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

    def get_response(self, user_input: str) -> str:
        """
        Generates a response based on the user input
        """
        # Simulate receiving the user input
        self.receive(name="User", message=user_input)

        # Generate a response using the chat model
        response = self.send()

        # Simulate receiving the response
        self.receive(name=self.name, message=response)

        return response


def send_message(vk, user_id, text):
    vk.method('messages.send', {'user_id': user_id, 'message': text, 'random_id': get_random_id()})


def main():
    vk_session = vk_api.VkApi(token=token)
    longpoll = VkLongPoll(vk_session)

    for event in longpoll.listen():
        if event.type == VkEventType.MESSAGE_NEW and event.to_me:
            handle_message(event)


def handle_message(event):
    vk = vk_api.VkApi(token=token)
    print('User input: ', event.text)

    # Get the response from the DialogueAgent
    response = party.get_response(event.text)

    print('output: ', response)
    send_message(vk, event.user_id, response)


# Instantiate the DialogueAgent
name = "Виртуальный ассистент Сириус"
party = DialogueAgent(name=name, system_message=system_message, model=giga)

# Run the main loop
main()
